In [13]:
#Importing Dependencies
!pip install tensorflow 
!pip install tensorflow-gpu 
!pip install opencv-python
!pip install mediapipe
!pip install --upgrade pip
!pip install sklearn 
!pip install matplotlib 



In [3]:
pip install --upgrade pip

     ---------------------------------------- 2.1/2.1 MB 10.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.3.1
    Uninstalling pip-22.3.1:
      Successfully uninstalled pip-22.3.1
Note: you may need to restart the kernel to use updated packages.


In [14]:
import cv2 
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from cv2 import VideoCapture

In [15]:
mp_holistic = mp.solutions.holistic  #Holistic
mp_drawing = mp.solutions.drawing_utils # Drawing Utitlities

In [16]:
def mediapipe_detection(image, model) :
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #Color conversion
    image.flags.writeable = False                  #Image is no longer writable 
    results = model.process(image)                 #make prediction
    image.flags.writeable = True                   #Image is writable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) #Color conversion back
    return image, results

In [17]:
def draw_landmarks(image, results) : 
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [18]:
def draw_styled_landmarks(image, results) : 
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

In [19]:
#Access the webcam
cap = VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic : 
    while cap.isOpened() :

        #Reading frames
        ret, frame = cap.read()

        #Make detection 
        image, results = mediapipe_detection(frame, holistic)
        draw_styled_landmarks(image, results)
        #print(results)

        #show to screen
        cv2.imshow('OpenCV Feed', image)

        #break 
        if cv2.waitKey(10) & 0xFF == ord('q') : 
            break

    cap.release()
    cv2.destroyAllWindows()

In [23]:
np.array(results.pose_landmarks.landmark).shape

(33,)

In [24]:
np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().shape

(132,)

In [25]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132*4)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [26]:
def extract_keypoints(results) : 
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [27]:
extract_keypoints(results).shape

(1662,)

In [28]:
results.face_landmarks

landmark {
  x: 0.5969406366348267
  y: 0.6595572233200073
  z: -0.01338899601250887
}
landmark {
  x: 0.6045300960540771
  y: 0.6287277340888977
  z: -0.04234171658754349
}
landmark {
  x: 0.5986237525939941
  y: 0.6368365287780762
  z: -0.01847064681351185
}
landmark {
  x: 0.5931740999221802
  y: 0.5847571492195129
  z: -0.04005148634314537
}
landmark {
  x: 0.6051016449928284
  y: 0.6162271499633789
  z: -0.04691632464528084
}
landmark {
  x: 0.6034350991249084
  y: 0.5980784893035889
  z: -0.046156786382198334
}
landmark {
  x: 0.5967372059822083
  y: 0.5526732206344604
  z: -0.03143405169248581
}
landmark {
  x: 0.5230328440666199
  y: 0.5483229160308838
  z: -0.015320144593715668
}
landmark {
  x: 0.5944315791130066
  y: 0.518378734588623
  z: -0.0308154858648777
}
landmark {
  x: 0.5945452451705933
  y: 0.5009526014328003
  z: -0.03576800227165222
}
landmark {
  x: 0.5914183855056763
  y: 0.4361686706542969
  z: -0.03923262283205986
}
landmark {
  x: 0.5964208245277405
  y: 0.6

In [29]:
#path for exported array
DATA_PATH = os.path.join('MP_Data')

#Actions that we are going to detect
actions = np.array(['hello', 'thanks', 'You`r Welcome'])

# 30 videos worth of data
no_sequences = 30 

#30 frames of length
sequence_length = 30 

In [30]:
for action in actions : 
    for sequence in range(no_sequences) : 
        try : 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except : 
            pass

In [31]:
cap = VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic : 
    #Loop through actions 
    for action in actions :
        #Loop through the collection of frames
        for sequence in range(no_sequences) : 
            
            #Loop through each frame
            for frame_num in range(sequence_length) : 
                
                #Reading frames
                ret, frame = cap.read()

                #Make detection 
                image, results = mediapipe_detection(frame, holistic)
                draw_styled_landmarks(image, results)
                #print(results)
                
                
                #collecting images 
                
                if frame_num == 0 : 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 1, cv2.LINE_AA)
                    cv2.putText(image, f'Collecting frames for {action} video number {sequence}', (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 4, cv2.LINE_AA)
                    cv2.waitKey(2000)
                else : 
                    cv2.putText(image, f'Collecting frames for {action} video number {sequence}', (15,12), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 4, cv2.LINE_AA)
                
                #show to screen
                cv2.imshow('OpenCV Feed', image)
                
                #Saving the arrays
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                #break 
                if cv2.waitKey(10) & 0xFF == ord('q') : 
                    break

    cap.release()
    cv2.destroyAllWindows()

In [32]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [33]:
label_map = {label : num for num, label in enumerate(actions)}

In [34]:
label_map

{'hello': 0, 'thanks': 1, 'You`r Welcome': 2}

In [35]:
#combining all sequences 

sequences = []
labels = []
for action in actions : 
    for sequence in range(no_sequences) : 
        window = []
        for frame_num in range(sequence_length) : 
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), f'{str(frame_num)}.npy'))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [36]:
np.array(sequences).shape

(90, 30, 1662)

In [37]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)

In [38]:
y

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0,

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05)

In [40]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [54]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [55]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation= 'relu', input_shape = (30,1662)))
model.add(LSTM(128, return_sequences=True, activation = 'relu'))
model.add(LSTM(64, return_sequences=False, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(actions.shape[0], activation = 'softmax'))

In [56]:
model.compile(optimizer= 'Adam', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])

In [57]:
model.fit(X_train, y_train , epochs = 245, callbacks=[tb_callback])

Epoch 1/245
3/3 [==============================] - 8s 218ms/step - loss: 1.5497 - categorical_accuracy: 0.5412
Epoch 2/245
3/3 [==============================] - 1s 177ms/step - loss: 2.4718 - categorical_accuracy: 0.4706
Epoch 3/245
3/3 [==============================] - 0s 161ms/step - loss: 4.2380 - categorical_accuracy: 0.2471
Epoch 4/245
3/3 [==============================] - 1s 179ms/step - loss: 2.0874 - categorical_accuracy: 0.2588
Epoch 5/245
3/3 [==============================] - 1s 188ms/step - loss: 1.1466 - categorical_accuracy: 0.3529
Epoch 6/245
3/3 [==============================] - 1s 214ms/step - loss: 1.0455 - categorical_accuracy: 0.3882
Epoch 7/245
3/3 [==============================] - 1s 230ms/step - loss: 0.9940 - categorical_accuracy: 0.4706
Epoch 8/245
3/3 [==============================] - 1s 197ms/step - loss: 0.8638 - categorical_accuracy: 0.6706
Epoch 9/245
3/3 [==============================] - 1s 204ms/step - loss: 0.6817 - categorical_accuracy: 0.7647
E

3/3 [==============================] - 1s 240ms/step - loss: 0.0011 - categorical_accuracy: 1.0000
Epoch 75/245
3/3 [==============================] - 0s 155ms/step - loss: 0.0010 - categorical_accuracy: 1.0000
Epoch 76/245
3/3 [==============================] - 1s 197ms/step - loss: 9.6189e-04 - categorical_accuracy: 1.0000
Epoch 77/245
3/3 [==============================] - 1s 191ms/step - loss: 8.8842e-04 - categorical_accuracy: 1.0000
Epoch 78/245
3/3 [==============================] - 1s 213ms/step - loss: 8.1717e-04 - categorical_accuracy: 1.0000
Epoch 79/245
3/3 [==============================] - 1s 193ms/step - loss: 7.7769e-04 - categorical_accuracy: 1.0000
Epoch 80/245
3/3 [==============================] - 1s 204ms/step - loss: 7.4964e-04 - categorical_accuracy: 1.0000
Epoch 81/245
3/3 [==============================] - 1s 194ms/step - loss: 7.1914e-04 - categorical_accuracy: 1.0000
Epoch 82/245
3/3 [==============================] - 1s 233ms/step - loss: 6.8771e-04 - catego

3/3 [==============================] - 1s 162ms/step - loss: 2.0598e-04 - categorical_accuracy: 1.0000
Epoch 145/245
3/3 [==============================] - 1s 209ms/step - loss: 2.0431e-04 - categorical_accuracy: 1.0000
Epoch 146/245
3/3 [==============================] - 1s 173ms/step - loss: 2.0100e-04 - categorical_accuracy: 1.0000
Epoch 147/245
3/3 [==============================] - 1s 230ms/step - loss: 1.9938e-04 - categorical_accuracy: 1.0000
Epoch 148/245
3/3 [==============================] - 1s 180ms/step - loss: 1.9594e-04 - categorical_accuracy: 1.0000
Epoch 149/245
3/3 [==============================] - 1s 246ms/step - loss: 1.9422e-04 - categorical_accuracy: 1.0000
Epoch 150/245
3/3 [==============================] - 1s 228ms/step - loss: 1.9138e-04 - categorical_accuracy: 1.0000
Epoch 151/245
3/3 [==============================] - 1s 201ms/step - loss: 1.8989e-04 - categorical_accuracy: 1.0000
Epoch 152/245
3/3 [==============================] - 1s 218ms/step - loss: 1.8

3/3 [==============================] - 1s 279ms/step - loss: 1.0322e-04 - categorical_accuracy: 1.0000
Epoch 215/245
3/3 [==============================] - 1s 208ms/step - loss: 1.0225e-04 - categorical_accuracy: 1.0000
Epoch 216/245
3/3 [==============================] - 1s 200ms/step - loss: 1.0170e-04 - categorical_accuracy: 1.0000
Epoch 217/245
3/3 [==============================] - 1s 195ms/step - loss: 1.0165e-04 - categorical_accuracy: 1.0000
Epoch 218/245
3/3 [==============================] - 1s 220ms/step - loss: 1.0052e-04 - categorical_accuracy: 1.0000
Epoch 219/245
3/3 [==============================] - 1s 210ms/step - loss: 9.9804e-05 - categorical_accuracy: 1.0000
Epoch 220/245
3/3 [==============================] - 1s 220ms/step - loss: 9.9261e-05 - categorical_accuracy: 1.0000
Epoch 221/245
3/3 [==============================] - 1s 198ms/step - loss: 9.8429e-05 - categorical_accuracy: 1.0000
Epoch 222/245
3/3 [==============================] - 1s 207ms/step - loss: 9.7

In [67]:
# evaluate the model accuarcy
loss, accuracy = model.evaluate(X_test, y_test)
print('Accuracy:', accuracy)

1/1 [==============================] - 1s 618ms/step - loss: 3.7193e-06 - categorical_accuracy: 1.0000
Accuracy: 1.0


In [45]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-trai

In [46]:
res= model.predict(X_test)

1/1 [==============================] - 1s 755ms/step


In [47]:
res

array([[9.9437606e-01, 1.4542965e-06, 5.6225448e-03],
       [9.9394894e-01, 1.3264035e-06, 6.0497476e-03],
       [9.9317217e-01, 1.8903634e-06, 6.8259588e-03],
       [3.8214703e-04, 6.8764073e-01, 3.1197712e-01],
       [3.1948596e-04, 7.1592319e-01, 2.8375742e-01]], dtype=float32)

In [58]:
actions[np.argmax(res[2])]

'hello'

In [59]:
actions[np.argmax(y_test[2])]

'hello'

In [60]:
model.save('action.h5')

In [51]:
# del model

In [61]:
model.load_weights('action.h5')

In [62]:
# res = []
sequence = []
sentence = []
threshold = 0.4

In [63]:
actions[np.argmax(res)]

'hello'

In [65]:
cap = VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic : 
    while cap.isOpened() :

        #Reading frames
        ret, frame = cap.read()

        #Make detection 
        image, results = mediapipe_detection(frame, holistic)
        draw_styled_landmarks(image, results)
        #print(results)
        
        #determining the result
        keypoints = extract_keypoints(results)
        sequence.insert(0, keypoints)
        sequence = sequence[:30]
        
        if len(sequence) == 30 :
            print(np.array(sequence).shape)
            res= model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])

            #SHOWING THE RESULTS

            #checking if the value of the predicted action is greater than the threshold
            if res[np.argmax(res)] > threshold: 

                #checking if the sentence has more than word
                if len(sentence) > 0: 

                    #if the predicted word doesn't match the last word, add it
                    if actions[np.argmax(res)] != sentence[-1] : 
                        sentence.append(actions[np.argmax(res)])

                    #if the predicted word matches the last word 
                else : 
                    sentence.append(actions[np.argmax(res)])

        #if the length of the sentence is greater than 5
        if len(sentence) > 5 : 

            #reduce the array to the last five values
            sentence = sentence[-5:]

        cv2.rectangle(image, (0,0), (640,40), (245,117,16) , -1)
        cv2.putText(image, ' '.join(sentence) , (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1 , (255,255,255), 1,  cv2.LINE_AA)

        #show to screen
        cv2.imshow('OpenCV Feed', image)


        #break 
        if cv2.waitKey(10) & 0xFF == ord('q') : 
            break

    cap.release()
    cv2.destroyAllWindows()

(30, 1662)
1/1 [==============================] - 0s 61ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 48ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 47ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 38ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 29ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 27ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 30ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 22ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 23ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 20ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 20ms/step
You`r Welcome
(30, 1662)
1/1 [==============================] - 0s 33ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 21ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 23ms/step
hello
(30, 1662)
1/1 [=========

1/1 [==============================] - 0s 32ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 29ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 35ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 32ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 20ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 47ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 20ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 32ms/step
You`r Welcome
(30, 1662)
1/1 [==============================] - 0s 28ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 21ms/step
You`r Welcome
(30, 1662)
1/1 [==============================] - 0s 24ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 21ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 20ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 22ms/step
hello
(30, 1662)
1/1 [=========

1/1 [==============================] - 0s 19ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 30ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 19ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 24ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 21ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 22ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 20ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 18ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 18ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 35ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 19ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 21ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 21ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 19ms/step
thanks
(30, 1662)
1/1 [===============

thanks
(30, 1662)
1/1 [==============================] - 0s 19ms/step
You`r Welcome
(30, 1662)
1/1 [==============================] - 0s 23ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 26ms/step
You`r Welcome
(30, 1662)
1/1 [==============================] - 0s 24ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 22ms/step
You`r Welcome
(30, 1662)
1/1 [==============================] - 0s 24ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 18ms/step
You`r Welcome
(30, 1662)
1/1 [==============================] - 0s 29ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 19ms/step
You`r Welcome
(30, 1662)
1/1 [==============================] - 0s 21ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 25ms/step
You`r Welcome
(30, 1662)
1/1 [==============================] - 0s 20ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 21ms/step
hello
(30, 1662)
1/1 [===========================

1/1 [==============================] - 0s 23ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 19ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 19ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 21ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 21ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 20ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 21ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 27ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 26ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 20ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 32ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 22ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 20ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 30ms/step
thanks
(30, 1662)
1/1 [===============

In [92]:
cap.release()
cv2.destroyAllWindows()